# Emotion Recognition

In this notebook, we'll see how we can usea packagge called fer to detect emotion in a video stream from the camera

In [2]:
%pip install fer

Note: you may need to restart the kernel to use updated packages.


### Do the necessary import

In [ ]:
import cv2 as cv
from fer import FER

### Create a function to show a label on an image

In [ ]:
def add_label(frame, label, x1, y1, x2, y2, color):
  if type(label) == 'str' and label.contains('\n'):
    label = label.split(',')
  frame = cv.rectangle(frame, (x1, y1), (x2, y2), color, 4)
  
  # The length of the longest label with 10 points per char is our lable hight 
  xl = len(max(label, key=len)) * 10
  frame = cv.rectangle(frame, (x1, y2), ( x1 + xl , y2 + (len(label) * 20) + 4), color, -1)
  for i, line in enumerate(label):
    frame = cv.putText(frame, line, (x1+2, y2+20 ), cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
    y2 = y2 + 20
  return frame

In [3]:
emotion_detector = FER(mtcnn=True)
backrounds = {
    'angry': (150,0,0),
    'disgust': (150,150,0),
    'fear': (100,100,100),
    'happy': (0,150,0),
    'sad': (0,0,150),
    'surprise': (0,150,150),
    'neutral': (0,0,0)
    }
backround = backrounds['neutral']

cap = cv.VideoCapture(0)
if not cap.isOpened():
 print("Cannot open camera")
 exit()
while True:
  # Capture frame-by-frame
  ret, frame = cap.read()
  # if frame is read correctly ret is True
  if not ret:
    print("Can't receive frame (stream end?). Exiting ...")
    break
  analysis = emotion_detector.detect_emotions(frame)  
  if analysis:
    for face in analysis:
      emotion = max(face['emotions'], key=face['emotions'].get)
      x1,y1,x2,y2 = face['box'][0]-50, face['box'][1]-80,face['box'][0] + face['box'][2]+50, face['box'][1] + face['box'][3]+50
      frame = add_label(frame, [emotion], x1, y1, x2, y2, backrounds[emotion])
  # Display the resulting frame
  cv.imshow('frame', frame)
  if cv.waitKey(1) == ord('q'):
    break
# When everything done, release the capture
cap.release()
cv.destroyAllWindows()